#Foundation


In [157]:
import pandas as pd
import numpy as np
import re

In [158]:
#chosenaggdfshortened['coding'] = np.random.choice(['general reporting', 'speculation', 'film info', 'box office', 'critical review', 'cultural impact'], size=len(chosenaggdfshortened))

In [159]:
#load actual df from csv
df = pd.read_csv('coded_sample.csv')

In [160]:
# Function to preprocess and fill empty 'coding' values
# def preprocess_and_fill_coding(row):
#     if pd.isnull(row['coding']) or row['coding'].strip() == '':
#         row['coding'] = np.random.choice(['general reporting', 'speculation', 'film info', 'box office', 'critical review', 'cultural impact'])
#     return row

# # Apply the preprocess_and_fill_coding function to each row
# df = df.apply(preprocess_and_fill_coding, axis=1)

In [161]:
df

,Unnamed: 0,Title,Description,label,coding
0,139,Greta Gerwig Breaks Global & US Box Office Rec...,"Barbie' crosses half-billion mark in U.S., whi...",tmnt,box office
1,84,Morally Uplifting TEENAGE MUTANT NINJA TURTLES...,TEENAGE MUTANT NINJA TURTLES: MUTANT MAYHEM ha...,tmnt,box office
2,251,Barbenheimer' tops $235 million in domestic de...,"""Barbenheimer"" exploded over the weekend, gene...",barbie,box office
3,68,Barbie vs Oppenheimer At Worldwide Box Office:...,Both Barbie and Oppenheimer remained strong an...,missionimpossible,box office
4,14,Mutant Mayhem Climbs Up From the Sewers and In...,‘Teenage Mutant Ninja Turtles: Mutant Mayhem’ ...,tmnt,box office
...,...,...,...,...,...
658,128,Haunted Mansion box office previews scare up a...,Disney's Haunted Mansion box office scares up ...,barbie,box office
659,112,Why TMNT: Mutant Mayhem Didn’t Use the Iconic ...,TMNT: Mutant Mayhem director Jeff Rowe opted t...,tmnt,film info
660,125,"Christopher Nolan on ‘Oppenheimer,’ Mr. Cartoo...","Christoper Nolan talks “Oppenheimer,” Mr. Cart...",oppenheimer,general reporting
661,77,Read America Ferrera's Full Barbie Movie Monol...,Gloria's speech about how it is impossible to ...,barbie,film info


In [162]:
# Function to fetch stopwords from URL
def fetch_stopwords(stopwords_url):
    stopwords = set()
    try:
        with urlopen(stopwords_url) as response:
            for line in response:
                line = line.decode('utf-8').strip()
                if not line.startswith('#') and line:
                    stopwords.add(line)
    except Exception as e:
        print(f"Error fetching stopwords from URL: {e}")
    return stopwords

# Fetch stopwords from URL
stopwords_url = 'https://gist.githubusercontent.com/larsyencken/1440509/raw/53273c6c202b35ef00194d06751d8ef630e53df2/stopwords.txt'
stopwords = fetch_stopwords(stopwords_url)

In [163]:
# Function to preprocess and remove stopwords from text
def preprocess_and_remove_stopwords(text, stopwords):
    replace_chars = set("()[]-.,?!:;#&")
    for char in replace_chars:
        text = text.replace(char, ' ')
    text = text.lower()
    
    # Remove stopwords
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    
    # Join the remaining words
    processed_text = ' '.join(filtered_words)
    
    return processed_text

# Apply the preprocess_and_remove_stopwords function to the 'description' column
df['Filtered Description'] = df['Description'].apply(lambda x: preprocess_and_remove_stopwords(x, stopwords))

In [164]:
df

,Unnamed: 0,Title,Description,label,coding,Filtered Description
0,139,Greta Gerwig Breaks Global & US Box Office Rec...,"Barbie' crosses half-billion mark in U.S., whi...",tmnt,box office,barbie' crosses half billion mark 'last voyage...
1,84,Morally Uplifting TEENAGE MUTANT NINJA TURTLES...,TEENAGE MUTANT NINJA TURTLES: MUTANT MAYHEM ha...,tmnt,box office,teenage mutant ninja turtles mutant mayhem mar...
2,251,Barbenheimer' tops $235 million in domestic de...,"""Barbenheimer"" exploded over the weekend, gene...",barbie,box office,"""barbenheimer"" exploded weekend generating est..."
3,68,Barbie vs Oppenheimer At Worldwide Box Office:...,Both Barbie and Oppenheimer remained strong an...,missionimpossible,box office,barbie oppenheimer remained strong achieved re...
4,14,Mutant Mayhem Climbs Up From the Sewers and In...,‘Teenage Mutant Ninja Turtles: Mutant Mayhem’ ...,tmnt,box office,‘teenage mutant ninja turtles mutant mayhem’ h...
...,...,...,...,...,...,...
658,128,Haunted Mansion box office previews scare up a...,Disney's Haunted Mansion box office scares up ...,barbie,box office,disney's haunted mansion box office scares hum...
659,112,Why TMNT: Mutant Mayhem Didn’t Use the Iconic ...,TMNT: Mutant Mayhem director Jeff Rowe opted t...,tmnt,film info,tmnt mutant mayhem director jeff rowe opted av...
660,125,"Christopher Nolan on ‘Oppenheimer,’ Mr. Cartoo...","Christoper Nolan talks “Oppenheimer,” Mr. Cart...",oppenheimer,general reporting,christoper nolan talks “oppenheimer ” cartoon ...
661,77,Read America Ferrera's Full Barbie Movie Monol...,Gloria's speech about how it is impossible to ...,barbie,film info,gloria's speech impossible woman talked scenes...


In [165]:
from collections import defaultdict

# Assuming your DataFrame is named df

# Function to count words for each coding topic
def count_words(df):
    word_counts = defaultdict(lambda: defaultdict(int))

    for index, row in df.iterrows():
        coding_topic = row['coding']
        description = row['Filtered Description']

        words = description.split()
        for word in words:
            if word.isalpha():
                word_counts[coding_topic][word] += 1

    sorted_word_counts = {coding_topic: dict(sorted(word_counts[coding_topic].items(), key=lambda x: x[1], reverse=True)) for coding_topic in word_counts}
                
    return sorted_word_counts

# Call the function with your DataFrame
sorted_word_counts_dict = count_words(df)

# # Display the word counts for each coding topic
# for coding_topic, word_counts in word_counts_dict.items():
#     print(f"Word counts for {coding_topic}: {word_counts}")

In [166]:
# Function to filter word counts based on frequency threshold
def filter_word_counts(word_counts, threshold):
    filtered_word_counts = defaultdict(lambda: defaultdict(int))
    
    for coding_topic, counts in word_counts.items():
        for word, count in counts.items():
            if count >= threshold:
                filtered_word_counts[coding_topic][word] = count
                
    return filtered_word_counts

# Assuming you already have a sorted_word_counts_dict
threshold = 5
filtered_word_counts_dict = filter_word_counts(sorted_word_counts_dict, threshold)

# Display the filtered word counts for each coding topic
for coding_topic, word_counts in filtered_word_counts_dict.items():
    print(f"Word counts for {coding_topic} (>= {threshold} counts): {word_counts}")

Word counts for box office (>= 5 counts): defaultdict(<class 'int'>, {'box': 60, 'office': 59, 'million': 32, 'weekend': 28, 'movie': 21, 'mutant': 18, 'barbie': 17, 'day': 16, 'impossible': 13, 'film': 13, 'ninja': 11, 'movies': 11, 'bros': 11, 'greta': 10, 'domestic': 10, 'warner': 10, 'billion': 9, 'oppenheimer': 9, 'tom': 9, 'biggest': 9, 'below': 9, 'turtles': 8, 'jones': 8, 'summer': 8, 'time': 7, 'sales': 7, 'christopher': 7, 'blue': 7, 'mission': 7, 'dial': 7, 'release': 7, 'world': 7, 'mark': 6, 'top': 6, 'grossing': 6, 'teenage': 6, 'china': 6, 'robbie': 6, 'beetle': 6, 'july': 6, 'weekends': 6, 'record': 6, 'half': 5, 'action': 5, 'worldwide': 5, 'ticket': 5, 'dead': 5, 'continued': 5, 'films': 5, 'earned': 5, 'look': 5, 'meg': 5, 'fourth': 5, 'hollywood': 5, 'margot': 5})
Word counts for critical review (>= 5 counts): defaultdict(<class 'int'>, {'mutant': 22, 'movie': 20, 'film': 20, 'jones': 16, 'turtles': 14, 'review': 11, 'ninja': 11, 'indiana': 11, 'dial': 11, 'mayhem':

In [167]:
import math

# Function to compute TF-IDF score
def compute_tfidf(tf, idf):
    return tf * idf

# Function to compute IDF (Inverse Document Frequency)
def compute_idf(total_codings, codings_with_word):
    return math.log(total_codings / codings_with_word)

In [168]:

# Function to compute most frequent and distinctive words for each coding
def compute_coding_language(coding_counts, num_words):
    coding_language = defaultdict(list)
    total_codings = len(coding_counts)

    # Goes over the codings and the word counts for each coding
    for coding_topic, word_counts in coding_counts.items():
        tfidf_scores = []

        # Compute TF-IDF scores for each word using the tfidf calculation functions
        for word, tf in word_counts.items():
            codings_with_word = sum(1 for other_coding, other_counts in coding_counts.items() if word in other_counts)
            idf = compute_idf(total_codings, codings_with_word)
            tfidf = compute_tfidf(tf, idf)
            tfidf_scores.append((word, tfidf))

        # Sort the words by TF-IDF score in descending order
        sorted_tfidf_scores = sorted(tfidf_scores, key=lambda x: x[1], reverse=True)

        # Extract the top num_words words for the coding
        top_words = [word for word, _ in sorted_tfidf_scores[:num_words]]
        coding_language[coding_topic] = top_words

    return coding_language


In [169]:
# Assuming you already have a filtered_word_counts_dict
num_words = 10  # Specify the number of top words you want to extract for each coding
coding_language_dict = compute_coding_language(filtered_word_counts_dict, num_words)

In [170]:
# Display the most frequent and distinctive words for each coding
for coding_topic, top_words in coding_language_dict.items():
    print(f"Top words for Coding '{coding_topic}': {top_words}")

Top words for Coding 'box office': ['box', 'office', 'million', 'weekend', 'domestic', 'billion', 'biggest', 'below', 'sales', 'bros']
Top words for Coding 'critical review': ['review', 'superhero', 'reviews', 'style', 'indiana', 'mayhem', 'jones', 'destiny', 'animated', 'harrison']
Top words for Coding 'cultural impact': ['banned', 'doll', 'christopher', 'film', 'movie', 'barbie']
Top words for Coding 'film info': ['director', 'jeff', 'rogen', 'impossible', 'rowe', 'seth', 'reckoning', 'cruise', 'james', 'actor']
Top words for Coding 'general reporting': ['director', 'watch', 'theaters', 'fans', 'movies', 'indiana', 'reckoning', 'week', 'august', 'upcoming']
Top words for Coding 'speculation': ['teases', 'return', 'dc', 'mutant', 'blue', 'beetle', 'ninja', 'turtles', 'movie', 'barbie']
